In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
path = '/content/gdrive/MyDrive/[광인사] 1차 프로젝트/코드정리/merged_file/'

In [3]:
%cd /content/gdrive/MyDrive/[광인사] 1차 프로젝트/코드정리/merged_file

/content/gdrive/.shortcut-targets-by-id/1rlRbTHMt0NUEyP8qWoMFBsBO1r3KNnsD/[광인사] 1차 프로젝트/코드정리/merged_file


In [4]:
import pandas as pd
import numpy as np
import requests
import json
from bs4 import BeautifulSoup
import time
import datetime
import pytz
import glob
import os
import unicodedata
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
from xgboost import plot_importance, plot_tree
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [5]:
!pip install pycaret -q #파이케럿 설치
!pip install markupsafe==2.1.1 -q #html markup #문자열의 이스케이프 처리를 안전하게 해주는 패키지 설치
!pip install Jinja2 #파일을 html로 만들어주는 패키지 설치
from pycaret.regression import * #파이케럿의 분류 모델을 모두 임포트

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.4/484.4 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.4/153.4 kB 16.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.8/106.8 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.4/73.4 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 82.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 80.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 80.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 104.9 MB/s eta 0:0

# 아이템별 가격 최소값, 평균값, 중앙값, 최대값 출력 ( 조건 달아서 출력, 개수가 몇개 이상 (최대값/최소값)이 몇 미만 등등 아이템만

In [6]:
file_list = os.listdir(path)
uni = unicodedata.normalize('NFD','merged.csv')
file_list = [file for file in file_list if file.find(uni)!=-1]
print(file_list)

['한기_merged.csv', '무결점 골든 베릴_merged.csv', '시브의 보조장비 보주_merged.csv', '왜곡된 차원의 큐브[교환가능]_merged.csv', '진정한 각성을 이룬 자_merged.csv', '초전도 에너지 코어 상자[1회 교환가능]_merged.csv', '백룡의 뿔_merged.csv', '균열의 단편_merged.csv', '무결점 라이언 코어_merged.csv', '견고한 수호자의 모래시계_merged.csv', '무결점 조화의 결정체_merged.csv', '수호자의 모래시계_merged.csv', '기민한 흑룡의 날개_merged.csv', '차오르는 수호자의 망토_merged.csv', '눈부신 황혼의 공명_merged.csv', '차오르는 수호자의 모래시계_merged.csv', '날카로운 흑룡의 날개_merged.csv', '기운찬 적룡의 뿔_merged.csv', '기운찬 백룡의 뿔_merged.csv', '황금빛 황혼의 공명_merged.csv', '조화로운 백룡의 뿔_merged.csv', '조화로운 흑룡의 뿔_merged.csv', '조화로운 적룡의 뿔_merged.csv', '칠흑같은 황혼의 공명_merged.csv', '기민한 적룡의 날개_merged.csv', '조화로운 청룡의 뿔_merged.csv', '빛을 머금은

데이터 전처리

In [7]:
# 데이터 불러오기
df = pd.read_csv("수호자의 모래시계_merged.csv")

In [8]:
# sldDate컬럼을 datetime형식으로 변환
df["soldDate"] = pd.to_datetime(df["soldDate"]).apply(lambda dt: datetime.datetime(dt.year, dt.month, dt.day,dt.hour,0))

In [9]:
df = df[["soldDate", "unitPrice"]]

In [10]:
# soldDate 단위(1시간 or 30분단위 등등) 평균값
df = df.groupby(["soldDate"], as_index=False).mean()
print(df.head())

             soldDate    unitPrice
0 2023-08-01 11:00:00  1852.222222
1 2023-08-01 12:00:00  1662.761905
2 2023-08-01 13:00:00  1862.357143
3 2023-08-01 14:00:00  1830.911765
4 2023-08-01 15:00:00  1771.193548


# df의 새로운 컬럼 prePrice에 이전 행의 unitPrice 넣기

In [11]:
df["prePrice"] = np.NaN
for i in range(1,len(df)):
  df.loc[i,"prePrice"] = df.loc[i-1,"unitPrice"]

In [12]:
# df 데이터 컬럼 추가 및 삭제
df['month'] = df['soldDate'].dt.month
df['day'] = df['soldDate'].dt.day
df['hour'] = df['soldDate'].dt.hour
print(df.head())

             soldDate    unitPrice     prePrice  month  day  hour
0 2023-08-01 11:00:00  1852.222222          NaN      8    1    11
1 2023-08-01 12:00:00  1662.761905  1852.222222      8    1    12
2 2023-08-01 13:00:00  1862.357143  1662.761905      8    1    13
3 2023-08-01 14:00:00  1830.911765  1862.357143      8    1    14
4 2023-08-01 15:00:00  1771.193548  1830.911765      8    1    15


In [13]:
# 특정 일시 기준으로 train, test split
date_time_str = '2023-08-08 00:00:00'
split =  datetime.datetime.strptime(date_time_str, '%Y-%m-%d %H:%M:%S')
df_train = df[df.soldDate<split]
df_test = df[df.soldDate>split]
df.drop(columns=['soldDate'], inplace=True)
df_train.drop(columns=['soldDate'], inplace=True)
df_test.drop(columns=['soldDate'], inplace=True)
print(df.head(3))
print(df_train.head(3))
print(df_test.head(3))

     unitPrice     prePrice  month  day  hour
0  1852.222222          NaN      8    1    11
1  1662.761905  1852.222222      8    1    12
2  1862.357143  1662.761905      8    1    13
     unitPrice     prePrice  month  day  hour
0  1852.222222          NaN      8    1    11
1  1662.761905  1852.222222      8    1    12
2  1862.357143  1662.761905      8    1    13
     unitPrice     prePrice  month  day  hour
152   9249.750  5947.987179      8    8     4
153   8503.125  9249.750000      8    8     5
154   7824.600  8503.125000      8    8     6


In [14]:
X_train = df_train[df_train.columns.difference(['unitPrice'])]
X_test = df_test[df_test.columns.difference(['unitPrice'])]
y_train = df_train['unitPrice']
y_test = df_test['unitPrice']

# 파이캐럿

In [15]:
df

,unitPrice,prePrice,month,day,hour
0,1852.222222,NaN,8,1,11
1,1662.761905,1852.222222,8,1,12
2,1862.357143,1662.761905,8,1,13
3,1830.911765,1862.357143,8,1,14
4,1771.193548,1830.911765,8,1,15
...,...,...,...,...,...
271,5437.020000,5512.400000,8,13,15
272,4671.187500,5437.020000,8,13,16
273,5257.404255,4671.187500,8,13,17
274,5298.536082,5257.404255,8,13,18


In [16]:
sup = setup(df, target='unitPrice', train_size = 0.8)

,Description,Value
0,Session id,6366
1,Target,unitPrice
2,Target type,Regression
3,Original data shape,"(276, 5)"
4,Transformed data shape,"(276, 5)"
5,Transformed train set shape,"(220, 5)"
6,Transformed test set shape,"(56, 5)"
7,Numeric features,4
8,Rows with missing values,0.4%
9,Preprocess,True


In [20]:
comp = compare_models(sort= 'R2', include=['huber', 'lasso', 'ridge', 'lr', 'lightgbm', 'knn', 'rf', 'gbr', 'xgboost', 'ada', 'dt'])

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
huber,Huber Regressor,795.1647,3150664.6387,1467.1644,0.7130,0.2435,0.1579,0.1370
lasso,Lasso Regression,854.5461,3122935.9969,1472.7476,0.7102,0.2709,0.1998,0.1490
ridge,Ridge Regression,854.5592,3122982.4625,1472.7645,0.7102,0.2709,0.1998,0.0870
lr,Linear Regression,854.5762,3122979.2281,1472.7695,0.7102,0.2708,0.1997,0.0610
lightgbm,Light Gradient Boosting Machine,970.5409,3774372.0990,1678.9951,0.6356,0.3039,0.2150,0.1010
knn,K Neighbors Regressor,996.5139,3881774.2750,1784.2603,0.5791,0.3319,0.2334,0.0680
rf,Random Forest Regressor,989.9331,4032284.1635,1792.6892,0.5588,0.2946,0.2060,0.1350
gbr,Gradient Boosting Regressor,968.6743,4049154.7072,1793.1112,0.5299,0.2923,0.2016,0.1200
xgboost,Extreme Gradient Boosting,1097.8618,5446782.8812,2050.9093,0.4530,0.3225,0.2398,0.0650
ada,AdaBoost Regressor,1208.4275,5723384.8331,2167.2290,0.3488,0.3697,0.3227,0.0920


Processing:   0%|          | 0/49 [00:00<?, ?it/s]

In [29]:
from sklearn.linear_model import HuberRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

default_huber_model = HuberRegressor()
tuned_huber_model = tune_model(default_huber_model, n_iter=100, optimize='R2')

default_xgb_model = XGBRegressor()
tuned_xgb_model = tune_model(default_xgb_model, n_iter=100, optimize='R2')

default_lgbm_model = LGBMRegressor()
tuned_lgbm_model = tune_model(default_lgbm_model, n_iter=100, optimize='R2')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,685.9739,1278555.6166,1130.7323,0.7569,0.2216,0.1612
1,936.8553,1794289.4219,1339.5109,0.8581,0.2210,0.1927
2,792.1487,1158324.4000,1076.2548,0.8122,0.1957,0.1749
3,1478.4774,13194353.1653,3632.4032,0.1193,0.5087,0.1942
4,1180.3604,10048045.2599,3169.8652,0.2783,0.3299,0.1525
5,534.5132,638166.3064,798.8531,0.8718,0.1248,0.0940
6,680.8351,1183797.9319,1088.0248,0.8643,0.3001,0.2462
7,375.7639,261116.1223,510.9952,0.9354,0.1129,0.0925
8,684.1791,800804.9361,894.8771,0.7565,0.2541,0.1713


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 10 folds for each of 100 candidates, totalling 1000 fits


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,655.9334,1462654.7500,1209.4026,0.7219,0.1971,0.1298
1,1137.8527,3605117.0000,1898.7146,0.7148,0.2812,0.2163
2,954.9785,1371558.5000,1171.1356,0.7777,0.2671,0.2478
3,1401.6871,13208049.0000,3634.2878,0.1184,0.5005,0.1536
4,1048.0741,8876236.0000,2979.3013,0.3624,0.2876,0.1337
5,705.6501,1007086.5625,1003.5370,0.7977,0.1806,0.1429
6,676.3615,1102787.1250,1050.1367,0.8736,0.2846,0.2293
7,527.2867,483830.6875,695.5794,0.8802,0.2024,0.1587
8,822.8422,996910.7500,998.4542,0.6969,0.3119,0.2302


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 10 folds for each of 100 candidates, totalling 1000 fits


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,749.2048,1616304.7429,1271.3397,0.6927,0.2201,0.1619
1,1221.2715,4170456.5031,2042.1696,0.6701,0.3444,0.2810
2,910.7343,1856962.4123,1362.7041,0.6990,0.2994,0.2528
3,1416.8569,11703705.2825,3421.0679,0.2188,0.4234,0.1879
4,1083.7686,7487229.8142,2736.2803,0.4622,0.2476,0.1373
5,784.2733,974600.0866,987.2184,0.8042,0.2194,0.1833
6,663.3324,1030381.9488,1015.0773,0.8819,0.2663,0.2142
7,555.3889,531581.5269,729.0964,0.8684,0.2010,0.1606
8,1006.8107,1440576.1733,1200.2400,0.5620,0.3265,0.2515


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 10 folds for each of 100 candidates, totalling 1000 fits


In [30]:
save_model(tuned_huber_model, '/content/gdrive/MyDrive/[광인사] 1차 프로젝트/코드정리/huber')
save_model(tuned_xgb_model, '/content/gdrive/MyDrive/[광인사] 1차 프로젝트/코드정리/xgb')
save_model(tuned_lgbm_model, '/content/gdrive/MyDrive/[광인사] 1차 프로젝트/코드정리/lgbm')

Transformation Pipeline and Model Successfully Saved
Transformation Pipeline and Model Successfully Saved
Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=FastMemory(location=/tmp/joblib),
          steps=[('numerical_imputer',
                  TransformerWrapper(include=['prePrice', 'month', 'day',
                                              'hour'],
                                     transformer=SimpleImputer())),
                 ('categorical_imputer',
                  TransformerWrapper(include=[],
                                     transformer=SimpleImputer(strategy='most_frequent'))),
                 ('trained_model',
                  LGBMRegressor(bagging_fraction=0.6, bagging_freq=3,
                                feature_fraction=0.6, min_child_samples=11,
                                min_split_gain=0.8, num_leaves=80, reg_alpha=5,
                                reg_lambda=0.01))]),
 '/content/gdrive/MyDrive/[광인사] 1차 프로젝트/코드정리/lgbm.pkl')

In [31]:
load_huber = load_model('/content/gdrive/MyDrive/[광인사] 1차 프로젝트/코드정리/huber')
load_xgb = load_model('/content/gdrive/MyDrive/[광인사] 1차 프로젝트/코드정리/xgb')
load_lgbm = load_model('/content/gdrive/MyDrive/[광인사] 1차 프로젝트/코드정리/lgbm')

Transformation Pipeline and Model Successfully Loaded
Transformation Pipeline and Model Successfully Loaded
Transformation Pipeline and Model Successfully Loaded


In [32]:
print(load_huber['trained_model'])
print(load_xgb['trained_model'])
print(load_lgbm['trained_model'])

HuberRegressor(alpha=1e-07, epsilon=1.8)
XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.5, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.05, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=11, max_leaves=None,
             min_child_weight=3, missing=nan, monotone_constraints=None,
             n_estimators=130, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)
LGBMRegressor(bagging_fraction=0.6, bagging_freq=3, feature_fraction=0.6,
              min_child_samples=11, min_split_gain=0.8, num_leaves=80,
              reg_alpha=5, reg_lambda=0.01)
